In [1]:
import osqp
import cvxpy as cp
import numpy as np

import scipy as sp
from scipy import sparse

import matplotlib.pyplot as plt
from scipy.linalg import hankel

import gym
import mujoco_py
from gym import wrappers
from IPython import display

%matplotlib inline

In [2]:
env = gym.make('InvertedPendulum-v4', render_mode='rgb_array')
obs = env.reset()

ud = []
yd = []
done = False
for i in range(27):
    action = env.action_space.sample()
    obs, rew, done, info, _ = env.step(action)
    ud.append(action)
    yd.append(obs)

env.close()

ud = np.array(ud)
yd = np.array(yd)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [3]:
T_ini = 4
n = 4
N = 5
env.close()
T = len(ud)

print(f'yd shape : {yd.shape}   ud shape : {ud.shape}')

L = T_ini + N + n
H = hankel(ud[0:L], ud[L:])
print(f'H has rank {np.linalg.matrix_rank(H)} and shape {H.shape}')

yd shape : (27, 4)   ud shape : (27, 1)
H has rank 13 and shape (13, 14)


In [4]:
L = T_ini + N
yd = yd.reshape((27*4,))

U = hankel(ud[0:L], ud[L:])
Y = np.zeros((L*4, 27-L+1))

for i in range(0,T-L+1):
    Y[:,i] = yd[4*i:4*(L+i)]

Up = U[0:T_ini, :]
Uf = U[T_ini:, :]
print(f'U = {U.shape}, Up = {Up.shape}, Uf = {Uf.shape}')

Yp = Y[0:4*T_ini, :]
Yf = Y[4*T_ini:, :]
print(f'Y = {Y.shape}, Yp = {Yp.shape}, Yf = {Yf.shape}')

u_ini = ud[3:3+T_ini].reshape((T_ini,))
y_ini = yd[12:12+4*T_ini].reshape((4*T_ini,))


U = (9, 18), Up = (4, 18), Uf = (5, 18)
Y = (36, 19), Yp = (16, 19), Yf = (20, 19)


In [5]:
g = cp.Variable(T - T_ini - N)
u = cp.Variable(N)
y = cp.Variable(N*4)
sig_y = cp.Variable(4*T_ini)

Q = np.eye(N*4)
R = np.eye(N)*0.01
x_lower = np.ones(N)*0
x_upper = np.ones(N)*3

u_constraint = np.ones(N)*3

cost = cp.quad_form(y, Q) + cp.quad_form(u,R) + cp.norm1(g)*10 + cp.norm1(sig_y)*0.01

env = gym.make('InvertedPendulum-v4', render_mode='rgb_array')
obs = env.reset()
done = False
yT = [*y_ini]
uT = [*u_ini]
# print(yT)

for i in range(30):

  constraints = [
    Up @ g == u_ini,
    Yp @ g == y_ini + sig_y,
    Uf @ g == u,
    Yf @ g == y,
    # A@g == b,
    cp.abs(u) <= u_constraint
    # cp.abs(u[1::2]) <= steer_constraint,
    # y <= x_upper,
    # y >= x_lower
  ]

  prob = cp.Problem(cp.Minimize(cost), constraints)
  prob.solve(solver=cp.OSQP, verbose=False)

  # for i in range(T_ini):
  plt.imshow(env.render())
  display.display(plt.gcf())    
  display.clear_output(wait=True)
  action = prob.variables()[1].value[0]
  # print(action)
  obs,_,_,_,_ = env.step([action])
  uT.append(action)
  for o in obs:
    yT.append(o) 
    
  # print(yT)
  u_ini = np.array(uT[-4:]).reshape(4,)
  y_ini = np.array(yT[-16:]).reshape(16,)





ValueError: Incompatible dimensions (16, 19) (18, 1)

In [ ]:
# time = np.array(list(range(0,N)))
# y = prob.variables()[0].value
# u = prob.variables()[1].value
yT = np.array(yT)
print(yT.shape)
yT = yT.reshape((34,4))
plt.subplot(1,2,1)
plt.plot(range(0,34), yT, label=['x-pos', 'angle', 'vel-x', 'ang-vel'])
plt.legend(loc="upper right")
plt.xlabel('Time (s)')
plt.title("X - Pos Prediction")
# plt.ylim((-3,7))
plt.grid('on')
plt.tight_layout()

plt.subplot(1,2,2)
plt.plot(range(0,34), uT)
plt.legend(loc="upper left")
plt.xlabel('Time (s)')
plt.title("Actuation")
plt.ylim((-3,7))
plt.grid('on')
plt.tight_layout()

plt.show()